## An exploration of piq and HINT footprints on chr21

I reported that our piq footprints (for 18 samples and ~550 motifs) totaled > 2B, but HINT footprints totaled only about 20M: two orders of magnitude.   Seth helpfully explained that we should expect to see a pretty good match of HINT and piq results if we filtered the piq results on some one or more of the scores piq reports.  I proposed that I do some exploratory (visual) data analysis of this matter, asked Seth for some thresholds.   Seth's reply:

<code>
Two approaches seem reasonable for filtering the PIQ results to get a smaller number of high-confidence footprints.

1. The authors of PIQ recommend performing filtering based on the purity score. "Purity" is an odd term for 1 - FDR. i.e., a purity score of 0.7 corresponds to a 30% (predicted) False Discovery Rate. I think this is fine, but I'm not sure I trust their purity score will really function as an FDR...

2. Alternatively, we can use the comparison to ChIP-seq to guide us.  It is common to define a cutoff value for the predictor that achieves a balance between precision and recall (i.e., the "precision-recall break even" point). Note that we can apply this approach to scores from Wellington and HINT, as well as PIQ. I calculated the PRBE for each of the three footprinting algorithms. For PIQ, I used the "score" column in my analysis, since this combines evidence about both the motif sequence and the shape of the DNase. The PRBE point will differ between TFs, so we'll have to see if there is a reasonable value to use across all the TFs with ChIP-seq. Here are the PRBE values for ELF1:

         AUROC              PRBE        Precision.PRBE
fimo    0.6566       14.62802959     0.124315387000103
well.   0.8938       19.14902115     0.591500347532519
hint    0.9742       228.1427216     0.710662422916252
piq     0.7833       40.51518076     0.5462218250422
</code>

This notebook
   * makes connections to temporary databases (chr21 only) for HINT and piq
   * gets some summary statistics
   * displays comparision igv tracks conditioned on region and scores
   

In [1]:
import pandas as pd, psycopg2 as psql
from igv import IGV, Reference, Track
import time

### make connections to the (currently separate) databases.  they wil be merged soon.

In [2]:
dbHint = psql.connect("dbname=hintTest2  user=pshannon")

In [3]:
dbpiq = psql.connect("dbname=piqTest user=pshannon")

### each database has two tables: regions, and hits.  learn their sizes before filtering

In [4]:
regionCount = pd.read_sql_query("select count(*) from regions", dbHint).ix[0,0]
hitCount = pd.read_sql_query("select count(*) from hits", dbHint).ix[0,0]
print("HINT: %d hits in %d regions, density %4.2f" % (hitCount, regionCount, 
                                                      hitCount/regionCount))

HINT: 504863 hits in 222660 regions, density 2.27


In [5]:
regionCount = pd.read_sql_query("select count(*) from regions", dbpiq).ix[0,0]
hitCount = pd.read_sql_query("select count(*) from hits", dbpiq).ix[0,0]
print("piq: %d hits in %d regions, density %4.2f" % (hitCount, regionCount, 
                                                     hitCount/regionCount))

piq: 30072096 hits in 1158121 regions, density 25.97


### An arbitrary choice: look upstream from the tss of the APP gene

In [6]:
APP_tss = 25880550
loc_chrom = "chr21"
loc_start = APP_tss - 3000
loc_stop  = APP_tss + 0

In [7]:
igv = IGV(locus="chr21:25,863,045-25,887,052", reference=Reference(id="hg38"), 
          tracks=[Track(
                   name="Genes hg38 v24",
                   format="gtf",
                   url="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz",
                   indexURL="//s3.amazonaws.com/igv.broadinstitute.org/annotations/hg38/genes/gencode.v24.annotation.sorted.gtf.gz.tbi",
                   visibility_window=10000000,
                   display_mode="SQUISHED")])


In [8]:
igv

### define a function which encapsulates the sql join, returns a table in bed format

In [9]:
def getHitsOld(db, chrom, start, end, scoreColumnNameToUse):
  query = """select * from regions r
    inner join hits h on r.loc = h.loc
    where r.chrom = '%s' and r.start > %d and r.stop < %d""" % (chrom, start, end)
  tbl = pd.read_sql_query(query, db)
  #return(tbl) 
  tbl['desc']  = tbl.name + "|" + tbl.sample_id  + "|" + tbl[scoreColumnNameToUse].astype(str) 
  return(tbl[["chrom", "start", "stop", "desc", scoreColumnNameToUse, "strand"]])

In [10]:
def getHits(db, chrom, start, stop, scoreColumnNameToUse):

   queryRegions_0 = "select loc, chrom, start, stop from regions"
   queryRegions_1 = "where chrom='%s' and start >= %d and stop <= %d" % (chrom, start, stop)
   queryRegions = "%s %s" % (queryRegions_0, queryRegions_1)
   tbl_regions = pd.read_sql_query(queryRegions, db)
   locs = tbl_regions.loc[:, "loc"].tolist()

   queryHits = "select * from hits where loc in %s" % str(locs)
   queryHits = queryHits.replace("[", "(").replace("]", ")")
   tbl_hits = pd.read_sql_query(queryHits, db)
   tbl = pd.merge(tbl_regions, tbl_hits, on="loc")
   tbl['desc'] = tbl.name + "|" + tbl.sample_id  + "|" + tbl[scoreColumnNameToUse].astype(str) 
   return(tbl[["chrom", "start", "stop", "desc", scoreColumnNameToUse, "strand"]])


In [11]:
tbl_hint = getHits(dbHint, "chr21", APP_tss-2000, APP_tss+100, "score1")
tbl_hint

,chrom,start,stop,desc,score1,strand
0,chr21,25879515,25879523,MA0597.1|ENCSR000EJB|39.0,39.0,-
1,chr21,25879510,25879519,MA0620.1|ENCSR000EJB|39.0,39.0,+
2,chr21,25879513,25879523,MA0646.1|ENCSR000EJB|39.0,39.0,+
3,chr21,25879839,25879847,MA0738.1|ENCSR000EJB|13.0,13.0,+


In [12]:
tbl_piq = getHits(dbpiq, "chr21", APP_tss-2000, APP_tss+100, "score4")

In [13]:
tbl_piq.shape

(2034, 6)

In [14]:
tbl_piq.subset7 = tbl_piq[tbl_piq["score4"] > 0.7]
tbl_piq.subset8 = tbl_piq[tbl_piq["score4"] > 0.8]
print(tbl_piq.subset7.shape)
print(tbl_piq.subset8.shape)

(88, 6)
(16, 6)


In [15]:
tbl_piq.subset8

,chrom,start,stop,desc,score4,strand
52,chr21,25879709,25879717,MA0031.1|ENCSR000EMS|0.804172,0.804172,-
70,chr21,25879709,25879717,MA0157.2|ENCSR000EMS|0.815756,0.815756,-
88,chr21,25879709,25879717,MA0613.1|ENCSR000EMS|0.81565,0.815650,-
329,chr21,25878818,25878825,MA0081.1|ENCSR000EJB|0.814106,0.814106,-
664,chr21,25878645,25878656,MA0136.2|ENCSR000EMS|0.835804,0.835804,+
779,chr21,25879904,25879910,MA0442.1|ENCSR000EJB|0.812273,0.812273,-
851,chr21,25879329,25879344,MA0505.1|ENCSR000EJB|0.806787,0.806787,+
1157,chr21,25879610,25879624,MA0518.1|ENCSR000EJB|0.960852,0.960852,+
1161,chr21,25879610,25879624,MA0518.1|ENCSR000EJG|0.942001,0.942001,+
1247,chr21,25879515,25879524,MA0597.1|ENCSR000EJB|0.871249,0.871249,-


### create an igv track viewer with gene reference track, then one each for piq & hint

In [16]:
def createTrack(tblHits, trackName):
   tblHits.to_csv("%s.bed" % trackName, sep="\t", header=False, index=False)
   print("wrote %s.bed" % trackName)
   newTrack = Track(name=trackName, 
                 format="bed", 
                 indexed=False, 
                 url="http://whovian:9999/files/%s.bed" % trackName, 
                 display_mode='EXPANDED')
   return(newTrack)
   

In [17]:
x = createTrack(tbl_piq.subset8, "piq.subset8")

wrote piq.subset8.bed


In [18]:
igv.load_track(x)

Loading track into IGV.js


In [19]:
tbl_hint

,chrom,start,stop,desc,score1,strand
0,chr21,25879515,25879523,MA0597.1|ENCSR000EJB|39.0,39.0,-
1,chr21,25879510,25879519,MA0620.1|ENCSR000EJB|39.0,39.0,+
2,chr21,25879513,25879523,MA0646.1|ENCSR000EJB|39.0,39.0,+
3,chr21,25879839,25879847,MA0738.1|ENCSR000EJB|13.0,13.0,+


In [20]:
x2 = createTrack(tbl_hint, "HINT")
igv.load_track(x2)

wrote HINT.bed
Loading track into IGV.js
